# **LAB 5**
______

## **Zad 1**
> #### `np` - numer procesu
> #### `lp` - liczba procesów
> #### `n` - indeks w szeregu (od 1)

In [138]:
%%sh
cat > pi-mpi.c << EOF


#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int main(int argc, char *argv[]) {
	int i,np,lp;
	int tag=50;
	MPI_Status status;
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &np);
	MPI_Comm_size(MPI_COMM_WORLD, &lp);

	if (np == 0) {
		double suma = 0, s=0, n = 1.0;
    printf("\nObliczanie równoległe liczby PI z wzoru Leibniza dla %d procesów", lp);
		for(i=0;i<lp;i++)
		{
      MPI_Send(&n, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
      n++;
		}

		for(i=0;i<lp;i++)
		{
			MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
			suma+=s;
		}
		printf("\nWynik = %lf\n", 4 * suma );
	}
	else
	{
		double s;
    double n;

    MPI_Recv(&n, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);

		s = pow(-1.0, n-1) / (2*n-1);
		MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD);
	}
	MPI_Finalize();
}
EOF
mpicc pi-mpi.c -lm && mpirun -n 128 --allow-run-as-root a.out 


Obliczanie równoległe liczby PI z wzoru Leibniza dla 128 procesów
Wynik = 3.133780


## **Zad 2**

> #### `np` - numer procesu
> #### `lp` - liczba procesów
> #### `dx` - odległość między sąsiednimi punktami ( wysokość trapezu )
> #### `xp` - początek przedziału
> #### `xk` - koniec przedziału

Metoda *funkcja* - funkcja podcałkowa

In [41]:

%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

double funkcja(double x) {
	return x;
}

int main(int argc, char *argv[]) {
	int i,np,lp;
	int tag=50;
	MPI_Status status;
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &np);
	MPI_Comm_size(MPI_COMM_WORLD, &lp);

	if (np == 0) {
		double xp,xk,pole,s=0,dx;
		printf("\nObliczanie calki oznaczonej metoda trapezów dla funkcji f(x)=x");
		xp = 1.0;
		xk = 4.0;
		dx=(xk-xp)/(double)lp;

		// wysylanie wartosci ci xp i xk oraz dx do poszczegolnych procesow
		for(i=0;i<lp;i++)
		{
			MPI_Send(&xp, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
			MPI_Send(&xk, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
			MPI_Send(&dx, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
		}

	  // odbieranie oraz sumowanie wyników z wszystkich procesów
		for(i=0;i<lp;i++)
		{
			MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
			pole+=s;
		}
		printf("\nPole pod calka wynosi = %lf\n", dx * (pole + (funkcja(xp) + funkcja(xk))/2) );
	}
	else
	{
		double s,dx,xp,xk;

		// odbieranie wartości od procesu 0
		MPI_Recv(&xp, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
		MPI_Recv(&xk, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
		MPI_Recv(&dx, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);

		// obliczanie pola trapezu dla podanej funkcji
		s = funkcja(xp + np * dx); 

		//wysyłanie pola trapezu do procesu 0
		MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD);
	}
	MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 8 --allow-run-as-root a.out


Obliczanie calki oznaczonej metoda prostokątów dla funkcji f(x)=x
Pole pod calka wynosi = 7.875000
